In [1]:
import pandas as pd
from scipy.io import arff


In [2]:
arff_file1 = arff.loadarff('./bank.arff')
df1 = pd.DataFrame(arff_file1[0])

In [2]:
arff_file = arff.loadarff('./census.arff')
df = pd.DataFrame(arff_file[0])

In [13]:
print(df.columns)

Index(['att2', 'att3', 'att4', 'att5', 'att7', 'att8', 'att9', 'att10',
       'att11', 'att12', 'att13', 'att14', 'att15', 'att16', 'att20', 'att21',
       'att22', 'att23', 'att24', 'att26', 'att27', 'att28', 'att29', 'att30',
       'att32', 'att33', 'att34', 'att35', 'att36', 'att37', 'att38', 'att39',
       'att41', 'class'],
      dtype='object')


In [5]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

df_cleaned = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Replace '_' with '=' in the column names
df_cleaned.columns = df_cleaned.columns.str.replace('_', '=')

# Map the 'class' column to 0 and 1
df_cleaned['class'] = df_cleaned['class'].apply(lambda x: 0 if x == '--50000.' else 1)

# Identify categorical columns excluding 'class'
categorical_columns = df_cleaned.select_dtypes(include=['object']).columns.difference(['class'])

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the data
encoded_data = encoder.fit_transform(df_cleaned[categorical_columns])

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Drop the original categorical columns and concatenate the encoded columns with the 'class' column
df_encoded = df_cleaned.drop(columns=categorical_columns).reset_index(drop=True)
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

print(df_encoded.head())

/tmp/ipykernel_132204/152231209.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


   class  att10_Adm-support-including-clerical  att10_Armed-Forces  \
0      0                                   0.0                 0.0   
1      0                                   0.0                 0.0   
2      0                                   0.0                 0.0   
3      0                                   0.0                 0.0   
4      0                                   0.0                 0.0   

   att10_Executive-admin-and-managerial  att10_Farming-forestry-and-fishing  \
0                                   0.0                                 0.0   
1                                   0.0                                 0.0   
2                                   0.0                                 0.0   
3                                   0.0                                 0.0   
4                                   0.0                                 0.0   

   att10_Handlers-equip-cleaners-etc  \
0                                0.0   
1                       

In [7]:
# Remove the 'b' prefix and decode bytes to strings
df_encoded.iloc[:1000, -1].value_counts()

att9_Wholesale-trade
0.0    985
1.0     15
Name: count, dtype: int64

In [8]:
df_encoded.columns = df_encoded.columns.str.replace('_', '=')
print(df_encoded.columns)

Index(['class', 'att10=Adm-support-including-clerical', 'att10=Armed-Forces',
       'att10=Executive-admin-and-managerial',
       'att10=Farming-forestry-and-fishing',
       'att10=Handlers-equip-cleaners-etc',
       'att10=Machine-operators-assmblrs-&-inspctrs', 'att10=Not-in-universe',
       'att10=Other-service', 'att10=Precision-production-craft-&-repair',
       ...
       'att9=Not-in-universe-or-children', 'att9=Other-professional-services',
       'att9=Personal-services-except-private-HH',
       'att9=Private-household-services', 'att9=Public-administration',
       'att9=Retail-trade', 'att9=Social-services', 'att9=Transportation',
       'att9=Utilities-and-sanitary-services', 'att9=Wholesale-trade'],
      dtype='object', length=504)


In [9]:

df_encoded.to_csv('census_encoded.csv', index=False)